In [1]:
# Essentials
import time
import copy
from collections import OrderedDict
import random
import os
from tifffile import TiffFile
from PIL import Image, ImageOps
from pathlib import Path
from tqdm.notebook import tqdm
# Data
import numpy as np
import pandas as pd
# Plot
import matplotlib.pyplot as plt
# Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, Subset
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
# Torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
# segmentation_models_pytorch
import segmentation_models_pytorch as smp
# Albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Local 
from unet import UNet
from LCD import LandCoverData
from dataset import *
from train import *

LCD = LandCoverData()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cuda:0


# ⚠️Seed everything!
It's important to seed everything for reproducibility.

In [27]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [28]:
seed = 2021
seed_everything(seed)

# 📜 Set all variables here
In this cell, we define all hyperparameters that we will be using for the rest of the notebook. It helps to group them in one place so we can track them better.

In [32]:
MODEL = 'resnet18'
OPTIMIZER = 'adam'
NB_EPOCHS = 20
LEARNING_RATE = 0.01
BATCH_SIZE = 8
IN_CHANNELS = 4

In [33]:
f"{MODEL}_{NB_EPOCHS}_epochs_{LEARNING_RATE}_learningrate_{BATCH_SIZE}_batchsize_seed_{seed}"

'resnet18_20_epochs_0.01_learningrate_8_batchsize_seed_2021'

# Dataset

### Define custom transforms

In [50]:
train_transform = A.Compose([
    A.ToFloat(max_value=65535.0),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    A.FromFloat(max_value=65535.0),
    A.Normalize(mean=(0, 0, 0, 0), std=(1, 1, 1, 1), max_pixel_value=65535),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Normalize(mean=(0, 0, 0, 0), std=(1, 1, 1, 1), max_pixel_value=65535),
    ToTensorV2()
])

### Initiate datasets
Here we perform a train/validation split in otder to evaluate our model. We then feed them to the ImageSegementationDataset so to make the datasets.

In [51]:
train_dir = 'dataset/train'
test_dir = 'dataset/test'

train_idx, val_idx = train_val_dataset(train_dir, val_split=0.2)
train_set = ImageSegementationDataset(train_dir, in_channels=IN_CHANNELS, path_index=train_idx, mode='train', transforms=train_transform)
val_set = ImageSegementationDataset(train_dir, in_channels=IN_CHANNELS, path_index=val_idx, mode='valid', transforms=test_transform)
test_set = ImageSegementationDataset(test_dir, in_channels=IN_CHANNELS, mode='test', transforms=test_transform)

print("Train set contains", len(train_set), "elements")
print("Validation set contains", len(val_set), "elements")
print("Test set contains", len(test_set), "elements")

Train set contains 14792 elements
Validation set contains 3699 elements
Test set contains 5043 elements


In [54]:
loader_train = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
loader_valid = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
loader_test = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

data_sizes = {"train": len(loader_train), "valid": len(loader_valid)}
print("There are", data_sizes['train'], "batches in the training set")
print("There are", data_sizes['valid'], "batches in the validation set")

There are 1849 batches in the training set
There are 463 batches in the validation set


### Dice Loss

In [55]:
def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = (y_pred_f * y_true_f).sum()
    smooth = 0.0001
    return 1 - (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

def change_shape(y_true, y_pred, numLabels):  

    encoded_target = y_pred.data.clone().zero_()
    encoded_target[...] = 0
    encoded_target.scatter_(1, torch.tensor(y_true.unsqueeze(1), dtype=torch.int64), 1.)
    encoded_target = Variable(encoded_target)
    return encoded_target

def dice_coef_multilabel(y_pred, y_true, numLabels):
    dice=0
    y_true = change_shape(y_true, y_pred, numLabels)
    for index in range(numLabels):
        dice += dice_coef(y_true[:,index,:,:], y_pred[:,index,:,:])
    return dice/numLabels # taking average

### IoU Metric

In [56]:
def mIOU(label, pred, num_classes=10):
    iou_list = list()
    present_iou_list = list()

    pred = pred.view(-1)
    label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            intersection_now = (pred_inds[target_inds]).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)
        iou_list.append(iou_now)
    return np.mean(present_iou_list)

### KL Divergence

In [57]:
def epsilon_kl_divergence(y_true, y_pred):
    class_distribution_true = np.apply_along_axis(np.bincount, axis=1, arr=y_true.flatten(1), minlength=LCD.N_CLASSES)
    class_distribution_pred = np.apply_along_axis(np.bincount, axis=1, arr=y_pred.flatten(1), minlength=LCD.N_CLASSES)
    # Normalize to sum to 1  
    normalized_class_distribution_true = (class_distribution_true.T/class_distribution_true.sum(1)).T
    normalized_class_distribution_pred = (class_distribution_pred.T/class_distribution_pred.sum(1)).T
    # add a small constant for smoothness around 0
    normalized_class_distribution_true += 1e-7
    normalized_class_distribution_pred += 1e-7

    score = np.mean(np.sum(normalized_class_distribution_true * np.log(normalized_class_distribution_true / normalized_class_distribution_pred), 1))
    try:
        assert np.isfinite(score)
    except AssertionError as e:
        raise ValueError('score is NaN or infinite') from e
    return score

# Training the Model

In [58]:
def training(model, train_loader, valid_loader, data_sizes, epochs, optimizer, scheduler, title):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device", device)
    model.to(device)
    
    since = time.time()

    training_loss = []
    validation_loss = []
    num_workers = 1

    best_model = copy.deepcopy(model.state_dict())
    best_acc = 1000

    loaders = {"train": train_loader, "valid": valid_loader}
    step = 0

    class_weights = class_weight().to(device)
    criterion_1 = nn.CrossEntropyLoss(class_weights)
    criterion_2 = dice_coef_multilabel
    
    for epoch in range(1, epochs+1):
        print(f'Epoch {epoch}/{epochs}')
        print('-' * 10)

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_iou = 0
            running_kl_div = 0

            for image, mask in tqdm(loaders[phase]):
                image = image.to(device)
                mask = mask.to(device)

                

                with torch.set_grad_enabled(phase=='train'):

                    output = model(image)
                    _, preds = torch.max(output, 1)

                    #loss = criterion(output, torch.tensor(mask, dtype=torch.long, device=device).squeeze())
                    loss = 0.75*criterion_1(output, torch.tensor(mask, dtype=torch.long, device=device).squeeze())+criterion_2(output, mask.squeeze(), LCD.N_CLASSES)
                

                    if phase == "train":
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                running_iou += mIOU(mask, preds)
                running_kl_div += epsilon_kl_divergence(mask.cpu(), preds.cpu())

            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss/data_sizes[phase]
            epoch_iou = running_iou/data_sizes[phase]
            epoch_kl = running_kl_div/data_sizes[phase]
            if phase == 'train':
                training_loss.append(epoch_loss)
            else:
                validation_loss.append(epoch_loss)

            print('{} Loss: {:.4f} IoU: {:.4f} KL_div: {:.4f}'.format(phase, epoch_loss, epoch_iou, epoch_kl))
            
            if phase == 'valid' and epoch_kl < best_acc:
                best_acc = epoch_kl
                best_model = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(),"unet_timm-efficientnet-b3_3_channels.pt")

            
    # Plotting the validation loss and training loss
    print('validation loss: ' + str(validation_loss))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    model.load_state_dict(best_model)

    # plot the training and validation loss
    plt.figure()
    plt.plot(training_loss, 'b', label='Training Loss')
    plt.plot(validation_loss, 'r', label='Validation Loss')
    plt.title(title)
    plt.legend()
    plt.show() #Change title for every model

    return model

In [60]:
def train_model(data_dir, model, epochs, learning_rate):
    # Optimizing all parameters
    optimizer_ft = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    # Training the model
    title = 'Variations of the training and validation loss SGD'
    model_ft = training(model, loader_train, loader_valid, data_sizes, epochs, optimizer_ft, exp_lr_scheduler, title)

    return model_ft

# Pre-trained Model

In [61]:
unet_pre_trained = smp.Unet(encoder_name='resnet18',in_channels=IN_CHANNELS, classes=10, activation='softmax')

In [ ]:
unet_pre_trained = train_model('dataset/train', unet_pre_trained, epochs=NB_EPOCHS, learning_rate=LEARNING_RATE)

Device cuda:0
Epoch 1/20
----------


  0%|          | 0/1849 [00:00<?, ?it/s]

<ipython-input-58-bc74afc39743>:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = 0.75*criterion_1(output, torch.tensor(mask, dtype=torch.long, device=device).squeeze())+criterion_2(output, mask.squeeze(), LCD.N_CLASSES)
<ipython-input-55-78a706cb2fa7>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encoded_target.scatter_(1, torch.tensor(y_true.unsqueeze(1), dtype=torch.int64), 1.)


train Loss: 2.0650 IoU: 0.3571 KL_div: 0.4930


  0%|          | 0/463 [00:00<?, ?it/s]

...

  0%|          | 0/1849 [00:00<?, ?it/s]

train Loss: 1.7879 IoU: 0.5347 KL_div: 0.0610


  0%|          | 0/463 [00:00<?, ?it/s]

valid Loss: 1.7690 IoU: 0.5533 KL_div: 0.0627
Epoch 12/20
----------


  0%|          | 0/1849 [00:00<?, ?it/s]

In [ ]:
running_iou = 0
running_kl_div = 0
unet_pre_trained.eval()
for image, mask in tqdm(loader_valid):
    image = image.to(device)
    mask = mask.to(device)
    with torch.no_grad():

        output = unet_pre_trained(image)
        _, preds = torch.max(output, 1)

        running_iou += mIOU(mask, preds)
        running_kl_div += epsilon_kl_divergence(mask.cpu(), preds.cpu())


epoch_iou = running_iou/len(loader_valid)
epoch_kl = running_kl_div/len(loader_valid)

print('Validation IoU', epoch_iou, "Validation KL divergence", epoch_kl)

  0%|          | 0/463 [00:00<?, ?it/s]

In [23]:
torch.save(unet_pre_trained.state_dict(),"unet_timm-efficientnet-b3_62_epochs.pt")

# Submission time

In [24]:
def batch_distribution(y):
    class_distribution = np.apply_along_axis(np.bincount, axis=1, arr=y.flatten(1), minlength=LCD.N_CLASSES)
    # Normalize to sum to 1  
    return (class_distribution.T/class_distribution.sum(1)).T

In [26]:
sub_dict = {"sample_id":[], "no_data":[],"clouds":[],"artificial":[],"cultivated":[],"broadleaf":[],"coniferous":[],"herbaceous":[],"natural":[],"snow":[],"water":[]}
for image, path in tqdm(loader_test):
    image = image.to(device)
    with torch.no_grad():
        output = unet_pre_trained(image)
        _, preds = torch.max(output, 1)
        class_dis = batch_distribution(preds.cpu())
        sub_dict["sample_id"] += [int(p.split('.')[0]) for p in path]
        for key in LCD.CLASSES:
            sub_dict[key] += class_dis[:,LCD.CLASSES.index(key)].tolist()

  0%|          | 0/631 [00:00<?, ?it/s]

In [27]:
df_sub = pd.DataFrame.from_dict(sub_dict)
df_sub = df_sub.sort_values(by='sample_id')
df_sub.head()

,sample_id,no_data,clouds,artificial,cultivated,broadleaf,coniferous,herbaceous,natural,snow,water
3436,10087,0.0,0.0,0.007431,0.248520,0.196350,0.013412,0.528641,0.000122,0.0,0.005524
3073,10088,0.0,0.0,0.004990,0.410492,0.176392,0.002701,0.405426,0.000000,0.0,0.000000
497,10089,0.0,0.0,0.055969,0.259369,0.226807,0.024384,0.426941,0.000015,0.0,0.006516
1319,10090,0.0,0.0,0.009201,0.037643,0.517822,0.148590,0.285034,0.001709,0.0,0.000000
2484,10091,0.0,0.0,0.026031,0.363190,0.188400,0.024353,0.397568,0.000229,0.0,0.000229


In [28]:
df_sub.to_csv('submission.csv')

# Saving the model

In [ ]:
torch.save(model_ft.state_dict(),"unet.pt")

# Loading the model

In [ ]:
def loading_saved_model(model_name):
    """Loads the saved model"""
    model = unet
    model.load_state_dict(torch.load(model_name, map_location = device))
    model.eval()
    return model

model_loaded = loading_saved_model("unet.pt")